In [63]:
!pip install tensorflow-gpu

In [ ]:
!unzip /content/ds.zip

Archive:  /content/ds.zip
  inflating: Fully Processed Datasets/ValidationOnly.tsv  
  inflating: Fully Processed Datasets/Ty.txt  
  inflating: Fully Processed Datasets/FinalTrainingOnly.tsv  
  inflating: Fully Processed Datasets/FinalTest.tsv  


In [133]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import csv
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense, Embedding ,Bidirectional
from sklearn.model_selection import train_test_split
import torch
from sklearn.metrics import accuracy_score

In [134]:
d = pd.read_csv('/content/FinalTrainingOnly.tsv', sep="\t", header=None )

x=d[1]
y=d[2]
y=list(y)
x=list(x)

for i in range(len(x)):
  if type(x[i])==float:
    x[i]=" "
    y[i]= 1

y=np.array(y)

In [135]:
v = pd.read_csv('/content/ValidationOnly.tsv', sep="\t", header=None )

vx=v[1]
vy=v[2]
vy=list(vy)
vx=list(vx)

for i in range(len(vx)):
  if type(vx[i])==float:
    vx[i]=" "
    vy[i]= 1

vy=np.array(vy)

In [136]:
t = pd.read_csv('/content/FinalTest.tsv', sep="\t", header=None )
tx=list(t[1])
for i in range(len(tx)):
  if type(tx[i])==float:
    tx[i]=" "

In [137]:
ty = pd.read_csv('/content/Ty.txt', sep=",", header=None )
sentiment = []
for i in range(len(ty)):
    if ty[1][i]=='negative':
        sentiment.append(0)
    elif ty[1][i]=='neutral':
        sentiment.append(1)
    elif ty[1][i]=='positive':
        sentiment.append(2)

ty = np.array(sentiment)

In [138]:
tokenizer = Tokenizer(num_words=2500,split=' ')
tokenizer.fit_on_texts(x)

X= tokenizer.texts_to_sequences(x)
X = pad_sequences(X,maxlen=50)

VX= tokenizer.texts_to_sequences(vx)
VX = pad_sequences(VX,maxlen=50)


TX = tokenizer.texts_to_sequences(tx)
TX = pad_sequences(TX,maxlen=50)

In [139]:
vocab_size = 20000
embed_size = 128

In [140]:
# Model no 1 

ip = tf.keras.Input(shape=(X.shape[1]))
em = Embedding(vocab_size, embed_size,trainable=True)(ip)
x = Bidirectional(LSTM(128,activation='tanh', return_sequences=True, dropout=0.15, recurrent_dropout=0.15))(em) # Dimn shd be (None,200,128)
x = tf.keras.layers.Conv1D(64, kernel_size=3, padding='valid', kernel_initializer='glorot_uniform')(x)
avg_pool = tf.keras.layers.GlobalAveragePooling1D()(x)
max_pool = tf.keras.layers.GlobalMaxPool1D()(x)
x = tf.keras.layers.concatenate([avg_pool, max_pool])
x = Dense(3, activation='softmax')(x)
bi = tf.keras.Model(inputs = ip , outputs = x)
bi.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-07), metrics=['accuracy'])
# tf.keras.utils.plot_model(bi, 'my_first_model.png')
history1 = bi.fit(X, y, epochs=7, batch_size=128,verbose=2,validation_data=(VX,vy))

Epoch 1/7
115/115 - 69s - loss: 0.9037 - accuracy: 0.5613 - val_loss: 0.7734 - val_accuracy: 0.6473
Epoch 2/7
115/115 - 68s - loss: 0.7836 - accuracy: 0.6440 - val_loss: 0.7082 - val_accuracy: 0.7073
Epoch 3/7
115/115 - 68s - loss: 0.7125 - accuracy: 0.6891 - val_loss: 0.6491 - val_accuracy: 0.7407
Epoch 4/7
115/115 - 69s - loss: 0.6446 - accuracy: 0.7251 - val_loss: 0.5943 - val_accuracy: 0.7640
Epoch 5/7
115/115 - 68s - loss: 0.5990 - accuracy: 0.7485 - val_loss: 0.5370 - val_accuracy: 0.7960
Epoch 6/7
115/115 - 69s - loss: 0.4858 - accuracy: 0.8027 - val_loss: 0.5842 - val_accuracy: 0.7713
Epoch 7/7
115/115 - 68s - loss: 0.4485 - accuracy: 0.8184 - val_loss: 0.4332 - val_accuracy: 0.8510


In [141]:
test_loss, test_acc = bi.evaluate(TX,ty)
print("Test accuracy of BiLSTM model = " + str(test_acc) )

94/94 [==============================] - 4s 40ms/step - loss: 1.1535 - accuracy: 0.6070
Test accuracy of BiLSTM model = 0.6069999933242798


In [142]:
# Model no 2 

ip = tf.keras.Input(shape=(X.shape[1]))
x = Embedding(vocab_size, embed_size,trainable=True)(ip)
x = tf.keras.layers.Conv1D(64, kernel_size=3, padding='valid', kernel_initializer='glorot_uniform')(x)
x = tf.keras.layers.GlobalMaxPool1D()(x)
x = tf.reshape(x,(-1,1,64))
x = LSTM(64,activation='tanh', dropout=0.15, recurrent_dropout=0.15)(x)
x = Dense(3, activation='softmax')(x)
cl = tf.keras.Model(inputs=ip, outputs=x)
cl.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-07), metrics=['accuracy'])
history = cl.fit(X, y, epochs=5, batch_size=128,verbose=2,validation_data=(VX,vy))

Epoch 1/5
115/115 - 5s - loss: 0.8826 - accuracy: 0.5718 - val_loss: 0.7676 - val_accuracy: 0.6567
Epoch 2/5
115/115 - 5s - loss: 0.7613 - accuracy: 0.6605 - val_loss: 0.7092 - val_accuracy: 0.7120
Epoch 3/5
115/115 - 5s - loss: 0.6792 - accuracy: 0.7128 - val_loss: 0.6160 - val_accuracy: 0.7530
Epoch 4/5
115/115 - 5s - loss: 0.5910 - accuracy: 0.7581 - val_loss: 0.5360 - val_accuracy: 0.7963
Epoch 5/5
115/115 - 5s - loss: 0.5259 - accuracy: 0.7915 - val_loss: 0.5357 - val_accuracy: 0.8043


In [143]:
test_loss, test_acc = cl.evaluate(TX,ty)
print("Test accuracy = " + str(test_acc) )

94/94 [==============================] - 0s 4ms/step - loss: 0.9451 - accuracy: 0.6077
Test accuracy = 0.6076666712760925


In [144]:
# Model no 3

model = Sequential()
model.add(Embedding(vocab_size, embed_size, input_shape = (X.shape[1],)))
model.add(LSTM(units=264, activation='tanh'))
model.add(Dense(units=64,activation='tanh'))
model.add(Dropout(0.5))
model.add(Dense(units=3, activation='softmax'))

model.compile(optimizer='adam', loss="sparse_categorical_crossentropy", metrics = ['accuracy'])
history = model.fit(X, y, epochs=6, batch_size=128,verbose=2,validation_data=(VX,vy))

Epoch 1/6
115/115 - 7s - loss: 0.9187 - accuracy: 0.5437 - val_loss: 0.8050 - val_accuracy: 0.6373
Epoch 2/6
115/115 - 6s - loss: 0.7865 - accuracy: 0.6433 - val_loss: 0.7750 - val_accuracy: 0.6493
Epoch 3/6
115/115 - 6s - loss: 0.7455 - accuracy: 0.6714 - val_loss: 0.7474 - val_accuracy: 0.6827
Epoch 4/6
115/115 - 6s - loss: 0.6887 - accuracy: 0.7073 - val_loss: 0.6534 - val_accuracy: 0.7270
Epoch 5/6
115/115 - 6s - loss: 0.6396 - accuracy: 0.7294 - val_loss: 0.6260 - val_accuracy: 0.7533
Epoch 6/6
115/115 - 6s - loss: 0.5896 - accuracy: 0.7531 - val_loss: 0.5908 - val_accuracy: 0.7720


In [145]:
test_loss, test_acc = model.evaluate(TX,ty)
print("Test accuracy = " + str(test_acc) )

94/94 [==============================] - 1s 8ms/step - loss: 0.9321 - accuracy: 0.5953
Test accuracy = 0.5953333377838135


In [146]:
# Model no 4

ip = tf.keras.Input(shape=(X.shape[1]))
b = Embedding(vocab_size, embed_size,trainable=True)(ip)
c1 = tf.keras.layers.Conv1D(64, kernel_size=3, padding='valid', kernel_initializer='glorot_uniform')(b)
c2 = tf.keras.layers.Conv1D(64, kernel_size=4, padding='valid', kernel_initializer='glorot_uniform')(b)
c3 = tf.keras.layers.Conv1D(64, kernel_size=5, padding='valid', kernel_initializer='glorot_uniform')(b)
x = tf.keras.layers.concatenate([c1, c2, c3],axis=1)
x = tf.keras.layers.GlobalMaxPool1D()(x)
x = tf.reshape(x,(-1,1,64))
x = Dense(64,activation='tanh')(x)
x = Dense(3, activation='softmax')(x)
cnn = tf.keras.Model(inputs=ip, outputs=x)
cnn.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-07), metrics=['accuracy'])
cnn.summary()
history = cnn.fit(X, y, epochs=4, batch_size=128,verbose=2,validation_data=(VX,vy))

Model: "model_20"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_21 (InputLayer)           [(None, 50)]         0                                            
__________________________________________________________________________________________________
embedding_27 (Embedding)        (None, 50, 128)      2560000     input_21[0][0]                   
__________________________________________________________________________________________________
conv1d_32 (Conv1D)              (None, 48, 64)       24640       embedding_27[0][0]               
__________________________________________________________________________________________________
conv1d_33 (Conv1D)              (None, 47, 64)       32832       embedding_27[0][0]               
___________________________________________________________________________________________

In [147]:
test_loss, test_acc = cnn.evaluate(TX,ty)
print("Test accuracy = " + str(test_acc) )

94/94 [==============================] - 1s 6ms/step - loss: 1.0141 - accuracy: 0.6033
Test accuracy = 0.6033333539962769


In [148]:
x1 = bi.predict(TX)
x2 = cl.predict(TX)
x3 = model.predict(TX)
x4 = cnn.predict(TX)

In [149]:
x5 = bi.predict(VX)
x6 = cl.predict(VX)
x7 = model.predict(VX)
x8 = cnn.predict(VX)

In [150]:
x4=np.reshape(x4,(3000,3))

In [151]:
x8=np.reshape(x4,(3000,3))

In [152]:
a=[]
for i in range(len(x1)):
  b=[]
  for j in range(len(x1[i])):
    z = max(x1[i][j],x2[i][j],x3[i][j],x4[i][j] )
    b.append(z)
  a.append(b)
  b=[]


In [153]:
c=[]
for i in range(len(x5)):
  d=[]
  for j in range(len(x5[i])):
    z = max(x5[i][j],x6[i][j],x7[i][j],x8[i][j] )
    d.append(z)
  c.append(d)
  d=[]

In [154]:
a=np.array(a)
a=np.argmax(a,axis=1)

In [155]:
c=np.array(c)
c=np.argmax(c,axis=1)

In [156]:
test_acc = accuracy_score(ty,a)
print("Test accuracy = " + str(test_acc) )

Test accuracy = 0.617


In [157]:
val_acc = accuracy_score(vy,c)
print("Validation accuracy = " + str(val_acc) )

Validation accuracy = 0.7623333333333333
